# **Dash app** for SIVEP-Gripe 

## **Lib**

In [202]:
import os
import sys
sys.path.append("..")
sys.path.append(os.path.join("..", "..", "linkage-saude"))

In [203]:
import json
import numpy as np
import pandas as pd
import datetime as dt
from simpledbf import Dbf5
# -- data matching
from linkage_saude.matching import PLinkage
# -- database connection
from injectsus import WarehouseSUS
from injectsus.process_layer import ProcessBase, ProcessSivep

In [303]:
from jupyter_dash import JupyterDash
from dash import dcc, html, Dash, Input, Output, State, callback_context, dash_table
import dash_bootstrap_components as dbc
import dash_echarts

# Warehouse tests

In [268]:
# -- ingest DBF file
fname = "SRAGHOSPITALIZADO1930520_00.dbf"
basepath = os.path.join(os.environ["HOMEPATH"], "Documents", "data")
datapath = os.path.join(basepath, "SIVEP-GRIPE", "MILLENA_14JUN2023")

sivep_df = Dbf5(os.path.join(datapath, fname), codec="latin").to_dataframe()
sivep_df1 = sivep_df.copy()
print(f"# records: {sivep_df1.shape[0]}")

sivep_df1["DT_NASC"] = pd.to_datetime(sivep_df1["DT_NASC"], format="%d/%m/%Y", errors="coerce")
sivep_df1["DT_NOTIFIC"] = pd.to_datetime(sivep_df1["DT_NOTIFIC"], format="%d/%m/%Y", errors="coerce")
sivep_df1.sample(n=4)

# records: 5305


,DT_RES_AN,RES_AN,LAB_AN,CO_LAB_AN,POS_AN_FLU,TP_FLU_AN,POS_AN_OUT,AN_SARS2,AN_VSR,AN_PARA1,...,PAC_DSCBO,OUT_ANIM,DOR_ABD,FADIGA,PERD_OLFT,PERD_PALA,TOMO_RES,TOMO_OUT,DT_TOMO,TP_TES_AN
1637,23/03/2023,1,NaN,NaN,1,1,2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,2.0
3321,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2,2,2,2,0.0,NaN,NaN,0.0
2142,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0
487,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2,2,2,2,6.0,NaN,NaN,0.0


In [269]:
warehousepath = os.path.join(basepath, "DATASUS_WAREHOUSE", "datasus_pessoas.db")
engine_url = f"sqlite:///{warehousepath}"
warehouse = WarehouseSUS(engine_url)
engine = warehouse.db_init()

In [6]:
# Uncomment if you want to reset table
#warehouse.delete_table('sivep_gripe', is_sure=True, authkey="###!Y!.")

In [270]:
warehouse.insert('sivep_gripe', sivep_df1, batchsize=50, verbose=True)

Insertion of batch 1 of 107 ... error: (sqlite3.IntegrityError) UNIQUE constraint failed: sivep_gripe.ID_SIVEP ... done.
Insertion of batch 2 of 107 ... error: (sqlite3.IntegrityError) UNIQUE constraint failed: sivep_gripe.ID_SIVEP ... done.
Insertion of batch 3 of 107 ... error: (sqlite3.IntegrityError) UNIQUE constraint failed: sivep_gripe.ID_SIVEP ... done.
Insertion of batch 4 of 107 ... error: (sqlite3.IntegrityError) UNIQUE constraint failed: sivep_gripe.ID_SIVEP ... done.
Insertion of batch 5 of 107 ... error: (sqlite3.IntegrityError) UNIQUE constraint failed: sivep_gripe.ID_SIVEP ... done.
Insertion of batch 6 of 107 ... error: (sqlite3.IntegrityError) UNIQUE constraint failed: sivep_gripe.ID_SIVEP ... done.
Insertion of batch 7 of 107 ... error: (sqlite3.IntegrityError) UNIQUE constraint failed: sivep_gripe.ID_SIVEP ... done.
Insertion of batch 8 of 107 ... error: (sqlite3.IntegrityError) UNIQUE constraint failed: sivep_gripe.ID_SIVEP ... done.
Insertion of batch 9 of 107 ... 

In [271]:
# -- query by notification health unit
q_example = warehouse.query_where('sivep_gripe', value="2526638", colname="CNES") # SOPAI
print(f'# records retrieved: {len(q_example)}')
pd.DataFrame(q_example).sample(n=5)

# records retrieved: 3156


,ID_SIVEP,DATA_NOTIFICACAO,NOME_PACIENTE,DATA_NASCIMENTO,SEXO,NOME_MAE,LOGRADOURO,LOGRADOURO_NUMERO,BAIRRO_RESIDENCIA,MUNICIPIO_RESIDENCIA,CEP,CNS,CPF,CNES,CRIADO_EM,ATUALIZADO_EM
542,31677504260761,2023-02-27,ATHALLY RODRIGUES DA SILVA PEREIRA,2019-12-05,M,MAYARA ARAUJO RODRIGUES,PADRE CONSTANTINO,187,JACARECANGA,230440,60310400,708003849104427,None,2526638,2023-08-02 14:52:15.306178,2023-08-02 14:52:15.306178
2137,31683614898695,2023-05-09,AYLLA GABRIELLY DOS SANTOS GONZAGA,2023-01-12,F,GLEICIANE GOMES DOS SANTOS GONZAGA,RUA LUMINOSA,4413,GRANJA LISBOA,230440,None,000000000000000,None,2526638,2023-08-02 14:52:16.969731,2023-08-02 14:52:16.969731
2759,31685373312336,2023-05-29,ARTHUR RONALD SILVA FRANCA,2022-01-07,M,IVANA DE SOUSA SILVA FRANCA,RUA ANA DE SA PIRES,160,MANOEL DIAS BRANCO,230440,None,None,12218791382,2526638,2023-08-02 14:52:17.481765,2023-08-02 14:52:17.481765
599,31677819956378,2023-03-03,MARIA LIA SOUSA SANTOS,2021-11-08,F,CAROLINA SOUSA SANTOS,AV. CAPITAO HUGO BEZERRA,1756,BARROSO,230440,None,898006264263569,None,2526638,2023-08-02 14:52:15.354053,2023-08-02 14:52:15.354053
1446,31681406293869,2023-04-13,JOAO GABRIEL OLIVEIRA DE LIMA,2023-02-15,M,None,ARMANDO DALL'OLIO,686,ENGENHEIRO LUCIANO CAVALCANTE,230440,60813575,898006319185645,None,2526638,2023-08-02 14:52:16.311952,2023-08-02 14:52:16.311952


# Supporting functions

In [335]:
def get_search_base(table_name, cnes=None, year_range=None):
    '''
        ...
    '''
    basepath = os.path.join(os.environ["HOMEPATH"], "Documents", "data")
    dbpath = os.path.join(basepath, "DATASUS_WAREHOUSE", "datasus_pessoas.db")
    engine_url = f"sqlite:///{dbpath}"
    dbsus = WarehouseSUS(engine_url)
    engine = dbsus.db_init()
    
    qdata = None
    if cnes is None and year_range is None:
        qdata = pd.DataFrame( dbsus.query_all(table_name) )
    elif cnes is None and year_range is not None:
        period = ( dt.datetime(year_range[0], 1, 1), dt.datetime(year_range[-1], 12, 31) )
        qdata = pd.DataFrame( dbsus.query_period('sivep_gripe', date_col='DATA_NOTIFICACAO', period=period ) )
    elif cnes is not None and year_range is None:
        qdata = pd.DataFrame( dbsus.query_where('sivep_gripe', value=cnes, colname="CNES") )
    elif cnes is not None and year_range is not None:
        period = ( dt.datetime(year_range[0], 1, 1), dt.datetime(year_range[-1], 12, 31) )
        qdata = pd.DataFrame( dbsus.query_period('sivep_gripe', date_col='DATA_NOTIFICACAO', period=period ) )
        qdata = qdata[ (qdata["DATA_NOTIFICACAO"]>=period[0]) & (qdata["DATA_NOTIFICACAO"]<=period[1]) ]
    
    qdata = pd.DataFrame( dbsus.query_all(table_name) )
    objdata = ProcessSivep(qdata, field_id="ID_SIVEP")
    objdata.basic_standardize().specific_standardize()
    proc_data = objdata.data.copy()

    return proc_data

def search_person(record, table_name, cnes=None, year_range=None, field_id="ID"):
    
    record_df = pd.DataFrame(record).reset_index()
    base_fields = ["NOME_PACIENTE", "DATA_NASCIMENTO", "NOME_MAE"]
    if not all([ elem in record_df.columns for elem in base_fields ]):
        raise Exception()
    
    objdata = ProcessBase(record_df, field_id=field_id)
    objdata.basic_standardize().specific_standardize()
    record_df = objdata.data.copy()

    searchbase = get_search_base(table_name, cnes, year_range)
    linkage = PLinkage(left_df=record_df, right_df=searchbase, left_id=field_id, right_id='ID_SIVEP', env_folder=None)
    
    map_compare = {
        "nascimento_dia": ["exact"], "nascimento_mes": ["exact"], "nascimento_ano": ["exact"],
        "primeiro_nome_mae": ["string", None], "complemento_nome_mae": ["string", None],
        "primeiro_nome": ["string", None], "complemento_nome": ["string", None],
    }
    map_sum = {
        "SOMA": list(map_compare.keys()),
        "SOMA ESSENCIAL 1": ["nascimento_dia", "nascimento_mes", "nascimento_ano", "primeiro_nome", "complemento_nome"],
    }
    linkage.set_linkage(map_compare, string_method="damerau_levenshtein")
    linkage.perform_linkage("FONETICA_N", window=3, threshold=0.75)
    comp_matrix = linkage.comparison_matrix.copy()
    comp_matrix["SOMA"] = comp_matrix.apply(sum, axis=1)
    comp_matrix_1 = comp_matrix[comp_matrix["SOMA"]>0].reset_index()
    found_ids = comp_matrix_1["ID_SIVEP"]
    collect = []
    for cur_id in found_ids:
        res = warehouse.query_where('sivep_gripe', value=cur_id, colname='ID_SIVEP')
        collect += res
    
    df = 'No result'
    if collect: 
        df = pd.DataFrame(collect).drop(["CRIADO_EM", "ATUALIZADO_EM"], axis=1)
    
    return df

# Dash app

In [158]:
# supporting data
basepath = os.path.join(os.environ["HOMEPATH"], "Documents", "data")
bairropath = os.path.join(basepath, "BAIRROS_FORTALEZA")

bairros_lst = pd.read_excel(os.path.join(bairropath, "TABELA_COD_NOME_BAIRRO_2022_higor.xls"), sheet_name="COD_BAIRRO")["nome_bairro"].tolist()
print(len(bairros_lst))

121


## Definitions

In [318]:
# nome paciente
patient_name = html.Div(
    [
        dbc.Label("NOME DO PACIENTE", html_for="example-email"),
        dbc.Input(type="text", value='', id="nome-paciente"),
    ],
    className="mb-3",
)
# nascimento
birth_input = html.Div(
    [
        dbc.Label("DATA DE NASCIMENTO"),
        dbc.Input(
            type="text",
            id="data-nascimento",
            value='',
            placeholder="DD/MM/AAAA",
        ),
    ],
    className="mb-3",
)
# nome mãe
mother_name = html.Div(
    [
        dbc.Label("NOME DA MÃE", html_for="example-email"),
        dbc.Input(type="text", value='', id="nome-mae"),
    ],
    className="mb-3",
)
# cnes
cnes_input = html.Div(
    [
        dbc.Label("CNES", html_for="cnes"),
        dbc.Input(type="text", value='', id="cnes"),
        dbc.FormText(
            "'TODAS' para selecionar todas unidades", color="secondary"
        ),
    ],
    className="col-sm",
)

# período de procura
year_marks = {
        0: {'label': '2020', 'style': {'color': '#77b0b1'}},
        1: {'label': '2021'},
        2: {'label': '2022'},
        3: {'label': '2023', 'style': {'color': '#f50'}}
    }
year_range = html.Div([
    dbc.Label("PERÍODO DE BUSCA", html_for="ano"),
    dcc.RangeSlider(0, 3, 1, value=[2,3], id='ano-slider', marks=year_marks), # --> Last year selected
    html.Div(id='output-container-range-slider')
], className="col-sm")


# bairro residência
bairro_dropdown = dbc.DropdownMenu(
    [ dbc.DropdownMenuItem(x, id=f'bairro_{x}') for x in ["NENHUM"]+bairros_lst ],
    id='bairro-dropdown',
    label="NENHUM",
    className="col-sm",
    toggle_style={
        "textTransform": "uppercase",
    },
)
bairro_input = html.Div(
    [  
        dbc.Label("BAIRRO DE RESIDÊNCIA (FORMATADO)"), 
        bairro_dropdown 
    ],
    id='bairro-input',
    className="col-sm",
)

# cep
cep_input = html.Div(
    [
        dbc.Label("CEP de residência"),
        dbc.Input(type="text", value='', id="cep"),
        dbc.FormText(
            "Somente números", color="secondary"
        ),
    ],
    className="mb-3",
)

# cns
cns_input = html.Div(
    [
        dbc.Label("CNS do Paciente"),
        dbc.Input(type="text", value='', id="cns"),
        dbc.FormText(
            "Somente números", color="secondary"
        ),
    ],
    className="col-sm",
)

# cpf
cpf_input = html.Div(
    [
        dbc.Label("CPF do Paciente"),
        dbc.Input(type="text", value='', id="cpf"),
        dbc.FormText(
            "Somente números", color="secondary"
        ),
    ],
    className="col-sm",
)

# sexo
sexo_input = html.Div(
    [
        dbc.Label("Sexo"),
        dbc.DropdownMenu(
            [ dbc.DropdownMenuItem('M', id='sexo_M'), dbc.DropdownMenuItem('F', id='sexo_F'), dbc.DropdownMenuItem('I', id='sexo_I') ],
            id='sexo-dropdown',
            label="I",
            className="col-sm",
        )
    ],
    className="col-sm",
)

## Layout

In [344]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.MINTY])

app.layout = dbc.Container([
    # -------- ROW 1
    dbc.Row([
        dbc.Col([
            dbc.Form([patient_name])
        ], width=5),
        dbc.Col([
            dbc.Form([sexo_input])
        ], width=1),
        dbc.Col([
            dbc.Form([birth_input])
        ], width=2),
        dbc.Col([
            dbc.Form([cpf_input])
        ], width=2),
        dbc.Col([
            dbc.Form([cns_input])
        ], width=2)
    ]),
    # -------- ROW 2
    dbc.Row([
        dbc.Col([
            dbc.Form([mother_name])
        ]),
        dbc.Col([
            dbc.Form([bairro_input])
        ]),
        dbc.Col([
            dbc.Form([cep_input])
        ]),
        dbc.Col([
            dbc.Form([cnes_input])
        ])
    ]),
    # --------- ROW 3
    dbc.Row([
        dbc.Col([
            dbc.Form([year_range])
        ]),
        dbc.Col([
            #dbc.Form([birth_input])
        ])
    ]),
    # --------- ROW 4
    dbc.Row([
        html.Div([
            dbc.Button("Submeter dados", id="example-button", className="me-1", n_clicks=0),
            html.Span(id="example-output", style={"verticalAlign": "middle"}),
        ])
    ]),
    # --------- ROW 5
    dbc.Row([
        dbc.Col([
            html.Div(id='display-output'),
        ], width=1),
        dbc.Col([
            html.Div(id='display-matrix'),
            #dash_table.DataTable(
            #    id='comp_matrix',
            #    data=None,
            #    #columns=[{'id': c, 'name': c} for c in df.columns],
            #)
        ], width=10)
    ]),
    
], fluid=True)



## Callbacks

In [345]:
# Callbacks

# -- dropdown label
persist_state = dict({'bairro': 'NENHUM', 'sexo': 'I'})
@app.callback(
    Output('bairro-dropdown', 'label'),
    Output('sexo-dropdown', 'label'),
    #Output('display-output', 'children'),
    [Input(f'bairro_{x}', 'n_clicks') for x in ["NENHUM"]+bairros_lst]+[Input(f'sexo_{x}', 'n_clicks') for x in ['M', 'F', 'I']]
)
def dropdown_labels(*args):
    '''
    
    '''
    ctx = callback_context
    triggered_input = ctx.triggered[0]['prop_id'].split(".")[0]
    
    last_bairro, last_sexo = persist_state['bairro'], persist_state['sexo']
    if 'bairro_' in triggered_input:
        last_bairro = triggered_input.replace("bairro_", "")
        persist_state['bairro'] = last_bairro
    if 'sexo_' in triggered_input:
        last_sexo = triggered_input.replace("sexo_", "")
        persist_state['sexo'] = last_sexo
    
    ctx_msg = json.dumps({
        'output': [last_bairro, last_sexo],
        #'test': triggered_input,
        'states': ctx.states,
        'triggered': ctx.triggered,
        'inputs': ctx.inputs
    }, indent=2)
    
    old_return = html.Div([ html.Pre(ctx_msg) ])
    return last_bairro, last_sexo

#@app.callback(
#    Output('bairro-dropdown', 'label'),
#    [Input(x, 'n_clicks') for x in ["NENHUM"]+bairros_lst],
#)
#def bairro_placeholder(*args):
#    ctx = callback_context
#    if not ctx.triggered:
#        button_id = "NENHUM"
#    else:
#        button_id = ctx.triggered[0]["prop_id"].split(".")[0]
#    return button_id

#@app.callback(
#    Output('sexo-dropdown', 'label'),
#    [Input(f'sexo-{x}', 'n_clicks') for x in ['M', 'F', 'I']],
#)
#def sexo_placeholder(*args):
#    #ctx = callback_context
#    if not callback_context.triggered:
#        label = "I"
#    else:
#        label = callback_context.triggered[0]["prop_id"].split(".")[0]
#    return label


@app.callback(
    #Output(component_id='display-output', component_property='children'),
    Output(component_id='display-matrix', component_property='children'),
    Input(component_id='example-button', component_property='n_clicks'),
    State(component_id='nome-paciente', component_property='value'),
    State(component_id='data-nascimento', component_property='value'),
    State(component_id='nome-mae', component_property='value'),
    State(component_id='bairro-dropdown', component_property='label'),
    State(component_id='sexo-dropdown', component_property='label'),
    State(component_id='cep', component_property='value'),
    State(component_id='cns', component_property='value'),
    State(component_id='cpf', component_property='value'),
    State(component_id='cnes', component_property='value'),
    State(component_id='ano-slider', component_property='value'),
)
def display_input(nclicks, nome, dtnasc, nomemae, bairro, sexo, cep, cns, cpf, cnes, year_range):
    if nclicks>=1:
        if cnes=='' or cnes=='TODAS': cnes = None
        if cpf=='': cpf = None
        if cns=='': cns = None
        if bairro=='NENHUM': bairro = None
        if cep=='': cep = None
        year_map = {0: 2020, 1: 2021, 2: 2022, 3: 2023}
        period = [ year_map[year_range[0]], year_map[year_range[-1]] ]
        dtnasc_fmt = pd.to_datetime(dtnasc, format="%d/%m/%Y")
        if pd.isna(dtnasc_fmt): dtnasc_fmt = None
    
        display_output = json.dumps({
            'NOME_PACIENTE': nome, 'DATA_NASCIMENTO': dtnasc,
            'NOME_MAE': nomemae, 'SEXO': sexo, 'BAIRRO_RESIDENCIA': bairro,
            'CEP': cep, 'CNS': cns, 'CNES': cnes, 'PERIODO': period
        }, indent=2)
        
        record = {
            'ID': ["XAHDFA"],
            'NOME_PACIENTE': [nome], 'DATA_NASCIMENTO': [dtnasc_fmt],
            'NOME_MAE': [nomemae], 'SEXO': [sexo], 'BAIRRO_RESIDENCIA': [bairro],
            'CEP': [cep], 'CNS': [cns], "CPF": [cpf],
        }
        comp_matrix = search_person(record, 'sivep_gripe', cnes=cnes, year_range=period, field_id='ID')
        dtable = comp_matrix
        if type(comp_matrix)!=str: 
            dtable = dash_table.DataTable(
                                    id='comp_matrix',
                                    data=comp_matrix.to_dict('records'),
                                    columns=[{'id': c, 'name': c} for c in comp_matrix.columns],
                                    style_data={
                                        'whiteSpace': 'normal',
                                        'height': 'auto',
                                    }
                                )
        return dtable

In [346]:
if __name__=="__main__":
    app.run_server(debug=True, port=8063, mode='inline')

Dash is running on http://127.0.0.1:8063/



Number of pairs: 4
Number of pairs: 4
Number of pairs: 3
Number of pairs: 4
Number of pairs: 3


In [135]:
def 

AttributeError: 'JupyterDash' object has no attribute '_terminate_server_for_port'

### explore data

In [ ]:
def gen_randlist(num):
    return random.sample(range(num), 7)


def main():
    '''
    dash_echarts examples
    name: smooth line with echarts
    author: dameng <pingf0@gmail.com>
    '''
    app = dash.Dash(__name__)

    option =  {
        'xAxis': {
            'type': 'category',
            'data': ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
        },
        'yAxis': {
            'type': 'value'
        },
        'series': [{
            'data': gen_randlist(200),
            'type': 'line',
            'smooth': True
        }, {
            'data': gen_randlist(200),
            'type': 'line',
            'smooth': True
        }]
    } 

    app.layout = html.Div([
        dash_echarts.DashECharts(
            option = option,
            id='echarts',
            style={
                "width": '100vw',
                "height": '100vh',
            }
        ),
        dcc.Interval(id="interval", interval=1 * 1000, n_intervals=0),
    ])


    @app.callback(
        Output('echarts', 'option'),
        [Input('interval', 'n_intervals')])
    def update(n_intervals):
        if n_intervals == 0:
            raise PreventUpdate
        else:
            option['series'][0]['data'] = gen_randlist(200)
            option['series'][1]['data'] = gen_randlist(200)
        return option
    app.run_server(debug=True)

#if __name__ == '__main__':
#    main()

### app

In [4]:
chart_option = {
    'xAxis': {
        'type': 'category',
        'data': ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    },
    'yAxis': {
        'type': 'value'
    },
    'series': [
        {
            'data': gen_randlist(200),
            'type': 'line',
            'smooth': True
        },
        {
            'data': gen_randlist(200),
            'type': 'line',
            'smooth': True
        }
    ]
}

NameError: name 'gen_randlist' is not defined

In [32]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.MINTY])

app.layout = dbc.Container([
    # ROW 1
    dbc.Row([
        dbc.Col([
            dcc.Dropdown(id="country", 
                         options=[{'label': country, 'value': country} for country in poverty_data['Country Name'].unique()]),
            html.Br(),
            html.Div(id="report", style={'textAlign': 'center'}, className='p-3')
        ])
    ]),
    # ROW 2
    dbc.Row([
        dbc.Col([
            html.Div([
                dash_echarts.DashECharts(
                    option = chart_option,
                    id='teste-chart',
                    style = {
                        "width": '100vw',
                        "height": '100vh',
                    }
                ),
                dcc.Interval(id="interval", interval=1 * 1000, n_intervals=0),
            ])
        ])
    ])
    
    
], fluid=True)

In [34]:
@app.callback(
    Output('report', 'children'),
    Input('country', 'value')
)
def filter_data(country):
    '''
    
    '''
    if country is None:
        return ''
    
    filter_country = poverty_data['Country Name']==country
    filter_indicator = poverty_data['Indicator Name']=='Population, total'
    filtered_df = poverty_data[(filter_country) & (filter_indicator)]
    population = filtered_df.loc[:, '2010'].values[0]
    
    return [ html.H3(country), 
            f'The population of {country} in 2010 was {population:,.0f}.' ]

@app.callback(
    Output('echarts', 'option'),
    [Input('interval', 'n_intervals')]
)
def update(n_intervals):
    if n_intervals == 0:
        raise PreventUpdate
    else:
        option['series'][0]['data'] = gen_randlist(200)
        option['series'][1]['data'] = gen_randlist(200)
    return option

In [21]:
#if __name__=="__main__":
#    app.run_server(mode="inline")

In [13]:
app._terminate_server_for_port("localhost", 8050)

AttributeError: 'JupyterDash' object has no attribute '_terminate_server_for_port'